#### Format for uploading the data for the project:
#####  --- “day_type”,“year”,“month”, “day”,“hour”,“minute”,“ID”, “district”,“HWNum”,“HWDIR”,“type”,“length”,“samples”,“obs_perc”,“tot_flow”,“avg_occ”,“avg_speed”
#### 5-min interval
#### Sensor number: 400868
#### Threshold: 
#####  --- 0 for less than 20 perc
#####  --- 1 for between 20 perc and 40 perc
#####  --- 2 for 40 perc and above
#### Day Type:
#####  --- 1 for holiday
#####  --- 2 for weekend
#####  --- 3 for weekday
#####  --- 4 for day before holiday
#### Consider decision tree and bi-nomial logistic

#### The columns we want are
####    --- day_type(int), time(hour*100+minute), avg_occupancy
#### Madhuri - To Pull and Prepare two data sets --- training and test - DONE
#### Training dataset has 8064 rows
#### Test dataset has 5184 rows

In [3]:
import pandas as pd
import numpy as np

MKtestRDD = sc.textFile("FileStore/tables/ha42p3kl1497485936953/MKtestDF.csv")
MKtrainingRDD = sc.textFile("/FileStore/tables/m0nlr27z1497924341940/MKtrainingDF.csv")
CintrainingRDD= sc.textFile("/FileStore/tables/4byv2yff1497333673791/training_mon_tue.csv")
CinTestRDD = sc.textFile("/FileStore/tables/5bp87o0a1497333538641/test_mon_tue.csv")
YuetestRDD=sc.textFile("/FileStore/tables/ha42p3kl1497485936953/YuetestRDD.csv")
YuetrainRDD=sc.textFile("/FileStore/tables/ha42p3kl1497485936953/YuetrainRDD.csv")
RohanTrainingRDD = sc.textFile("FileStore/tables/RBtrainingDF.csv")
RohanTestRDD = sc.textFile("FileStore/tables/RBtestDF.csv")

MKtrainingRDD.take(500)
##CintrainingRDD.count()


In [4]:
RDDtrain = MKtrainingRDD.union(CintrainingRDD).union(YuetrainRDD).union(RohanTrainingRDD)
RDDtrain.count()

In [5]:
RDDtest = MKtestRDD.union(CinTestRDD).union(YuetestRDD).union(RohanTestRDD)
##RDDtrain.take(20)
RDDtest.count()

In [6]:
import pandas as pd
from pyspark.sql import Row

traint= RDDtrain.map(lambda x:x.split(",")).map(lambda x:x[0:17])
traint = traint.map(lambda x: (x[0],x[1],x[2],x[3],int(x[4]),x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15],x[16]))
traint.take(5)
trainDF = traint.toDF(["day_type","year","month","day","hour","minute","ID", "district","HWNum","HWDIR","type","length","samples","obs_perc","tot_flow","avg_occ","avg_speed"])
display(trainDF)

In [7]:
import pandas as pd
from pyspark.sql import Row

testt= RDDtest.map(lambda x:x.split(",")).map(lambda x:x[0:17])
testt = testt.map(lambda x: (x[0],x[1],x[2],x[3],int(x[4]),x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],float(x[15]),x[16]))
testt.take(5)
testDF = testt.toDF(["day_type","year","month","day","hour","minute","ID", "district","HWNum","HWDIR","type","length","samples","obs_perc","tot_flow","avg_occ","avg_speed"])
display(testDF)

In [8]:
trainingDF = trainDF.toPandas()
testingDF = testDF.toPandas()
trainingDF.to_csv('/dbfs/FileStore/tables/training.csv', sep=',', header=False, index=False)
testingDF.to_csv('/dbfs/FileStore/tables/test.csv', sep=',', header=False, index=False)
display(dbutils.fs.ls("/FileStore/tables/"))